# Affinity-based instance segmentation

Lorem ipsum

In [ ]:
# TODO google colab stuff

## Affinity prediction

Lorem ipsum

In [7]:
import bioimageio.core
import h5py
import napari
from bioimageio.core.prediction import predict_with_tiling
from xarray import DataArray

In [4]:
# load the model
# TODO from doi
model = "./NeuronEMSegmentationAffinityModel.zip"
model = bioimageio.core.load_resource_description(model)

torch-em.git: 208kiB [00:00, 3.04MiB/s]


In [5]:
# load the data
with h5py.File("./sample_A+_20160601.h5", "r") as f:
    raw = f["volumes/raw"][:]

In [24]:
# run prediction
# NOTE: needs to run on gpu

# use the default configuration from the model
# tiling = True
# custom tiling configuration
tiling = {"tile": {"x": 120, "y": 120, "z": 16}, "halo": {"x": 12, "y": 12, "z": 4}}

with bioimageio.core.create_prediction_pipeline(bioimageio_model=model) as pp:
    input_ = DataArray(raw[None, None], dims=tuple(pp.input_specs[0].axes))
    affinities = predict_with_tiling(pp, input_, verbose=True, tiling=tiling)
affinities = affinities[0].values[0]

prediction with tiling: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 968/968 [42:17<00:00,  2.62s/it]


In [18]:
# visualize the predictions
v = napari.Viewer()
v.add_image(raw)
v.add_image(affinities)

<Image layer 'Image' at 0x7f35f00afdf0>

## Segmentation with the Mutex Watershed

Lorem ipsum

In [20]:
import numpy as np
from elf.segmentation.mutex_watershed import mutex_watershed

In [27]:
offsets = model.config["mws"]["offsets"]
strides = [4, 4, 4]
# TODO run on full volume once we have merged the
bb = np.s_[25:50, :1024, :1024]
segmentation = mutex_watershed(affinities[(slice(None),) + bb], offsets, strides)

In [28]:
v = napari.Viewer()
v.add_image(raw[bb])
v.add_labels(segmentation)

<Labels layer 'segmentation' at 0x7f35ebf0da30>

In [ ]:
# save the segmentation
out_path = "sampleA+_segmentation.h5"
with h5py.File(out_path, "a") as f:
    f.create_dataset("segmentation", data=segmentation, compression="gzip")